# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import time
np.random.seed(97)

# Base Network

In [2]:
class Model:
    def __init__(self, layers):
        self.layers = layers
    
    def forward(self):
        for layer in layers:
            layer.forward()
    
    def backward(self):
        for layer in self.layers.reversed():
            layer.backward()

### BinaryClassifier with vector calculation

In [3]:
    
class BinaryClassifier:
    
    def __init__(self, DIM=(1,2)):
        self.DIM = DIM
        self.w = np.zeros(self.DIM, dtype=np.float64)
        self.b = 0

    def forward(self, X):
        MIN_MARGIN = 2 ** -53

        self.z = np.dot(self.w, X.T) + self.b
        self.a = 1 / (1 + np.exp(-self.z))
        self.a = np.maximum(np.minimum(1 - MIN_MARGIN, self.a), MIN_MARGIN)
        return self.a

    def backward(self, X, y):
        da = -y / self.a + (1 - y) / (1 - self.a) #(1,100)
        dz = self.a * (1 - self.a) * da           #(1,100)
        dw = np.mean(X * dz.T, 0)                 #(2)
        db = np.mean(1 * dz)                      #()
        return dw, db

    def train(self, X, y, learning_rate):
        self.forward(X)
        dw, db = self.backward(X, y)
        self.w -= learning_rate * dw
        self.b -= learning_rate * db

    def predict(self, X):
        return np.round(self.forward(X))

    def loss(self, X, y):
        pred_y = self.forward(X)
        return -np.mean(y * np.log(pred_y) + (1 - y) * np.log(1 - pred_y))


### BinaryClassifier with elementwise calculation

In [4]:
    
class BinaryClassifierElementwise:
    
    def __init__(self, DIM=(1,2)):
        self.DIM = DIM
        self.w = np.zeros(self.DIM, dtype=np.float64)
        self.b = 0
    
    def forward(self, X):
        MIN_MARGIN = 2 ** -53

        # self.z = np.dot(self.w, X.T) + self.b
        tmp_z = []
        tmp_a = []

        for i in range(X.shape[0]):
            tmp_z.append(self.w[0][0]*X[i][0] + self.w[0][1]*X[i][1] + self.b)
            tmp_a.append(1 / (1 + np.exp(-tmp_z[i])))
            
            tmp_min = min(1 - MIN_MARGIN, tmp_a[i])
            tmp_a[i] = tmp_min if tmp_min>MIN_MARGIN else MIN_MARGIN
        
        self.a = np.array(tmp_a).reshape((1, X.shape[0]))
        return self.a

    def backward(self, X, y):
        
        X_nums = X.shape[0]
        
        da=[[]] #(1,100)
        dz=[[]] #(1,100)
        dw=[0.0, 0.0] #(2)
        db=0.0 #()
        for i in range(X_nums):
            da[0].append(-y[i] / self.a[0][i] + (1 - y[i]) / (1 - self.a[0][i]))
            dz[0].append(self.a[0][i] * (1 - self.a[0][i]) * da[0][i])
            dw[0] += X[i][0] * dz[0][i]
            dw[1] += X[i][1] * dz[0][i]
            db += 1 * dz[0][i]
            
        dw[0]/=X_nums
        dw[1]/=X_nums
        db/=X_nums
        return np.array(dw), db

    def train(self, X, y, learning_rate):
        self.forward(X)
        dw, db = self.backward(X, y)
        self.w[0][0] -= learning_rate * dw[0]
        self.w[0][1] -= learning_rate * dw[1]
        self.b -= learning_rate * db

    def predict(self, X):
        return np.round(self.forward(X))

    def loss(self, X, y):
        pred_y = self.forward(X)
        return -np.mean(y * np.log(pred_y) + (1 - y) * np.log(1 - pred_y))


# Train Model

In [5]:
def create_samples(sample_num):
    X = np.random.randint(-10, 10, (sample_num, 2))
    y = (np.sum(X, 1) >0).astype(int)
    return X, y


In [6]:
def train(train_samples, test_samples, learning_rate, epochs):
    
    # model = Model([BinaryClassifier((1,2))])
    classifier = BinaryClassifier((1,2))
    train_X, train_y = create_samples(train_samples)
    test_X, test_y = create_samples(test_samples)

    for epoch in range(epochs):
        classifier.train(train_X, train_y, learning_rate)
    
    return {'w': classifier.w.reshape((1,2)), 
            'b': classifier.b,
            'train_loss': classifier.loss(train_X, train_y),
            'test_loss': classifier.loss(test_X, test_y),
            'train_acc': 100 * np.mean(classifier.predict(train_X) == train_y),
            'test_acc': 100 * np.mean(classifier.predict(test_X) == test_y)}

In [7]:
def train_elementwise(train_samples, test_samples, learning_rate, epochs):
    
    # model = Model([BinaryClassifier((1,2))])
    classifier = BinaryClassifierElementwise((1,2))
    train_X, train_y = create_samples(train_samples)
    test_X, test_y = create_samples(test_samples)

    for epoch in range(epochs):
        classifier.train(train_X, train_y, learning_rate)
    
    return {'w': classifier.w.reshape((1,2)), 
            'b': classifier.b,
            'train_loss': classifier.loss(train_X, train_y),
            'test_loss': classifier.loss(test_X, test_y),
            'train_acc': 100 * np.mean(classifier.predict(train_X) == train_y),
            'test_acc': 100 * np.mean(classifier.predict(test_X) == test_y)}

In [8]:
train_samples = 100 # m
test_samples = 100 # n
learning_rate = 1e-2
epochs = 100 # K

train(train_samples, test_samples, learning_rate, epochs)


{'w': array([[0.37411396, 0.45177562]]),
 'b': -0.018992888908487472,
 'train_loss': 0.17186290637380514,
 'test_loss': 0.1494035394228638,
 'train_acc': 96.0,
 'test_acc': 97.0}

# Time Comparison

### element-wise version

In [9]:
start=time.time()
result_elementwise = train_elementwise(train_samples, test_samples, learning_rate, epochs)
print(time.time()-start)
print(result_elementwise)

0.5377762317657471
{'w': array([[0.38266354, 0.43251565]]), 'b': -0.02747253406561454, 'train_loss': 0.17816258634077775, 'test_loss': 0.16564797215229432, 'train_acc': 99.0, 'test_acc': 100.0}


### vector version

In [10]:
start=time.time()
result_vector = train(train_samples, test_samples, learning_rate, epochs)
print(time.time()-start)
print(result_vector)

0.012678146362304688
{'w': array([[0.41454156, 0.40819955]]), 'b': -0.04905599544328503, 'train_loss': 0.1466250264640928, 'test_loss': 0.15289553856197846, 'train_acc': 98.0, 'test_acc': 100.0}


# Test Parameter

In [11]:
def test_parameter(train_samples, test_samples, learning_rate,  epochs, runs):
    results=[]
    for run in range(runs):
        results.append(train(train_samples, test_samples, learning_rate,  epochs))
    
    results_df = pd.DataFrame(results)[['train_loss', 'test_loss', 'train_acc', 'test_acc']]
    normalized_result = results_df.sum()/runs
    
    return normalized_result

In [12]:
runs = 10

test_parameter(train_samples, test_samples, learning_rate, epochs, runs)


train_loss     0.165415
test_loss      0.165954
train_acc     98.200000
test_acc      98.500000
dtype: float64

### Test Parameter m (=train_samples)

In [13]:
for m in [10,100,1000]:
    print("\nTrain_samples : ", m)
    print(test_parameter(m, test_samples, learning_rate, epochs, runs))



Train_samples :  10
train_loss     0.123799
test_loss      0.284132
train_acc     99.000000
test_acc      88.200000
dtype: float64

Train_samples :  100
train_loss     0.176211
test_loss      0.175695
train_acc     98.400000
test_acc      97.700000
dtype: float64

Train_samples :  1000
train_loss     0.171175
test_loss      0.162078
train_acc     98.630000
test_acc      99.200000
dtype: float64


### Test Parameter K (=epochs)

In [14]:
for K in [10,100,1000]:
    print("\nEpochs : ", K)
    print(test_parameter(train_samples, test_samples, learning_rate, K, runs))


Epochs :  10
train_loss     0.382770
test_loss      0.389212
train_acc     97.600000
test_acc      96.400000
dtype: float64

Epochs :  100
train_loss     0.175958
test_loss      0.165278
train_acc     97.600000
test_acc      97.100000
dtype: float64

Epochs :  1000
train_loss     0.078167
test_loss      0.076554
train_acc     99.600000
test_acc      98.900000
dtype: float64


### Test Parameter lr (=learning_rate)

In [15]:
for lr in [1e-8, 1e-4, 1e-2, 1e-1, 1e0, 1e1]:
    print("\nLearning_rate : ", lr)
    print(test_parameter(train_samples, test_samples, lr, epochs, runs))


Learning_rate :  1e-08
train_loss     0.693142
test_loss      0.693142
train_acc     96.400000
test_acc      95.600000
dtype: float64

Learning_rate :  0.0001
train_loss     0.637911
test_loss      0.641886
train_acc     95.500000
test_acc      93.000000
dtype: float64

Learning_rate :  0.01
train_loss     0.172011
test_loss      0.169323
train_acc     98.600000
test_acc      98.800000
dtype: float64

Learning_rate :  0.1
train_loss     0.07443
test_loss      0.07818
train_acc     99.70000
test_acc      99.70000
dtype: float64

Learning_rate :  1.0
train_loss      0.02046
test_loss       0.03349
train_acc     100.00000
test_acc       99.80000
dtype: float64

Learning_rate :  10.0
train_loss      0.000645
test_loss       0.008518
train_acc     100.000000
test_acc       99.800000
dtype: float64
